In [1]:
import os
import cv2
from tqdm import tqdm

input_dir = '../../Datasets/shanghaitech/testing/frames'
output_dir = '../../Datasets/shanghaitech/testing/clips'
os.makedirs(output_dir, exist_ok=True)

window_size = 24
stride = 12

def pad_num(n):
    # 根据你的需求自动补零，最多三位（如 001）
    return str(n).zfill(3)

for folder_name in tqdm(sorted(os.listdir(input_dir))):
    folder_path = os.path.join(input_dir, folder_name)
    if not os.path.isdir(folder_path):
        continue

    images = sorted([img for img in os.listdir(folder_path) if img.lower().endswith(('.jpg', '.png'))])
    if len(images) < window_size:
        print(f"文件夹 {folder_name} 不足 {window_size} 帧，跳过")
        continue

    # 读取第一帧确定分辨率
    first_frame = cv2.imread(os.path.join(folder_path, images[0]))
    height, width, _ = first_frame.shape
    fps = 24
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # 滑动窗口生成视频
    num_images = len(images)
    # for start_idx in range(0, num_images - window_size + 1, stride):
    #     end_idx = start_idx + window_size - 1
    #
    #     # 命名格式：[001_024]文件夹名.mp4
    #     start_str = pad_num(start_idx)
    #     end_str = pad_num(end_idx)
    #     video_name = f'{folder_name}-{start_str}_{end_str}.mp4'
    #     video_path = os.path.join(output_dir, video_name)
    #     out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))
    #
    #     for idx in range(start_idx, end_idx + 1):
    #         img_path = os.path.join(folder_path, images[idx])
    #         frame = cv2.imread(img_path)
    #         if frame is None:
    #             print(f"读取帧失败: {img_path}")
    #             continue
    #         out.write(frame)
    #     out.release()
    #     # print(f"合成完成: {video_path}")
    resized_height, resized_width = 476, 868  # 指定resize的高和宽
    for start_idx in range(0, num_images - window_size + 1, stride):
        end_idx = start_idx + window_size - 1

        # 命名格式：[001_024]文件夹名.mp4
        start_str = pad_num(start_idx)
        end_str = pad_num(end_idx)
        video_name = f'{folder_name}-{start_str}_{end_str}.mp4'
        video_path = os.path.join(output_dir, video_name)
        out = cv2.VideoWriter(video_path, fourcc, fps, (resized_width, resized_height))  # 这里也要改

        for idx in range(start_idx, end_idx + 1):
            img_path = os.path.join(folder_path, images[idx])
            frame = cv2.imread(img_path)
            if frame is None:
                print(f"读取帧失败: {img_path}")
                continue
            # 这里加resize
            frame = cv2.resize(frame, (resized_width, resized_height), interpolation=cv2.INTER_CUBIC)
            out.write(frame)
        out.release()


print("全部滑动窗口视频合成完成！")


100%|██████████| 107/107 [04:07<00:00,  2.32s/it]

全部滑动窗口视频合成完成！
